# Examine triangle inequality
20250219

Author: Kyoko Kusano

放置すれば全部できる。三角不等式の破れ度合いを計算している。
20250403現在、単に割合を表示。

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math

import os
import sys
from pathlib import Path

# フォルダパスを指定してsys.pathに追加
sys.path.append(str(Path(os.getcwd()).resolve().parent / "func"))

# 関数が格納されているモジュールをインポート
import amalysis as amalysis

In [2]:
raw_path = str(Path(os.getcwd()).resolve()) + '/data/Amy_dissimilarity.csv'
raw = pd.read_csv(raw_path, header=None).T
raw.index = ['word1'] + ['word2'] + list(range(1, raw.shape[0]-1))
raw = raw.rename(columns={0:'BDI'})
raw.loc['word2', 'BDI'] = np.nan
head = raw.iloc[:2, :].copy()

df_raw = raw.iloc[2:, :].copy()
df_raw = df_raw.sort_values('BDI', ignore_index=True)
df_raw = df_raw.drop('BDI', axis=1)
df_raw = df_raw # dissimilarity (make it similarity in exponential part)
df_raw = pd.concat([head, df_raw]).drop('BDI', axis=1).copy()
df_raw

,1,2,3,4,5,6,7,8,9,10,...,244,245,246,247,248,249,250,251,252,253
word1,brown,orange,red,black,green,red,envy,fear,pink,blue,...,blue,blue,fear,purple,disgust,calm,orange,green,brown,anxiety
word2,anger,anxiety,yellow,blue,calm,pink,joy,anger,green,anger,...,envy,anxiety,happiness,confidence,defeated,happiness,pink,disgust,joy,boredom
0,3,4,2,2,4,2,6,6,2,4,...,6,6,7,1,0,1,1,6,6,0
1,7,5,5,6,2,2,7,6,7,7,...,6,5,7,3,3,2,4,5,5,4
2,6,6,0,0,2,0,2,1,1,5,...,7,6,6,7,3,1,0,7,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,7,7,7,3,3,3,7,3,3,7,...,7,0,7,7,2,0,7,0,7,7
116,7,7,6,5,4,2,7,2,5,7,...,3,6,7,3,2,0,2,1,7,7
117,7,7,4,3,3,2,7,3,5,7,...,5,7,7,2,6,0,1,7,7,4
118,7,5,4,4,3,1,7,3,4,6,...,7,6,5,3,3,2,3,1,7,3


In [4]:
# Average over subjects
# make a "similarity matrix" in tlab: word1 and 2 are inordered, use unique word in case you didn't order it
unique_words = np.array(['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'brown', 'grey', 'black', 'happiness', 'joy', 'calm', 'confidence', 'envy', 'fear', 'anxiety', 'disgust', 'sadness', 'confusion', 'boredom', 'defeated', 'anger'])

df_dissim = amalysis.create_dissimilarity_matrix(df_raw, unique_words)

In [5]:
# dissimilarity そのものが距離として妥当か三角不等式を全部に適用して確かめる

res_arr = np.empty((23, 23*23))
res_arr[:, :] = np.nan


df_dissim_temp = df_dissim.copy()
for idx1, pair1 in enumerate(unique_words):
    #print(f"idx1:{idx1}")
    for idx2, pair2 in enumerate(unique_words):
        #print(f"idx2:{idx2}")
        row = idx1*23 + idx2
        for idx3, single in enumerate(unique_words):
            longest = df_dissim_temp.iloc[idx1, idx2]
            short1 = df_dissim_temp.iloc[idx1, idx3]
            short2 = df_dissim_temp.iloc[idx2, idx3]

            ans = (short1 + short2) - longest # 三角不等式が成立していたら正
            
            if ans < 0:
                
                res_arr[idx3, row] = 1 # 破れの度合いを知りたいので合計できるように
            else:
                res_arr[idx3, row] = 0

print(np.isnan(res_arr).any())
print(np.shape(res_arr))
print(np.sum(res_arr))
mean_ratio = np.sum(res_arr) / (23*23*23)
print(mean_ratio)

False
(23, 529)
92.0
0.007561436672967864


In [6]:
# each individual
unique_words = np.array(['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'brown', 'grey', 'black', 'happiness', 'joy', 'calm', 'confidence', 'envy', 'fear', 'anxiety', 'disgust', 'sadness', 'confusion', 'boredom', 'defeated', 'anger'])

df_dissim_allsub = []
for sub in range(len(df_raw.iloc[:,0])-2):
    df_dissim_temp = amalysis.create_dissimilarity_matrix(
        df_raw, unique_words, subjects_no=sub
    )
    df_dissim_allsub.append(df_dissim_temp)

In [4]:
# dissimilarity そのものが距離として妥当か三角不等式を全部に適用して確かめる
###  ！！！！時間かかる！！！！ ###

res_arr_allsub = np.empty((len(df_raw.iloc[:,0])-2, 23, 23*23))
res_arr_allsub[:, :, :] = np.nan

for sub in range(len(df_raw.iloc[:,0])-2):
    print(f"sub: {sub}")
    df_dissim_temp = df_dissim_allsub[sub]
    for idx1, pair1 in enumerate(unique_words):
        #print(f"idx1:{idx1}")
        for idx2, pair2 in enumerate(unique_words):
            #print(f"idx2:{idx2}")
            row = idx1*23 + idx2
            for idx3, single in enumerate(unique_words):
                longest = df_dissim_temp.iloc[idx1, idx2]
                short1 = df_dissim_temp.iloc[idx1, idx3]
                short2 = df_dissim_temp.iloc[idx2, idx3]
    
                ans = (short1 + short2) - longest # 三角不等式が成立していたら正
                
                if ans < 0:
                    
                    res_arr_allsub[sub, idx3, row] = 1 # 破れの度合いを知りたいので合計できるように
                else:
                    res_arr_allsub[sub, idx3, row] = 0   

'\nres_arr = np.empty((len(df_raw.iloc[:,0])-2, 23, 23*23))\nres_arr[:, :, :] = np.nan\n\nfor sub in range(len(df_raw.iloc[:,0])-2):\n    print(f"sub: {sub}")\n    df_dissim_temp = df_dissim_allsub[sub]\n    for idx1, pair1 in enumerate(unique_words):\n        #print(f"idx1:{idx1}")\n        for idx2, pair2 in enumerate(unique_words):\n            #print(f"idx2:{idx2}")\n            row = idx1*23 + idx2\n            for idx3, single in enumerate(unique_words):\n                longest = df_dissim_temp.iloc[idx1, idx2]\n                short1 = df_dissim_temp.iloc[idx1, idx3]\n                short2 = df_dissim_temp.iloc[idx2, idx3]\n    \n                ans = (short1 + short2) - longest # 三角不等式が成立していたら正\n                \n                if ans < 0:\n                    \n                    res_arr[sub, idx3, row] = 1 # 破れの度合いを知りたいので合計できるように\n                else:\n                    res_arr[sub, idx3, row] = 0\n'

In [5]:
print(np.isnan(res_arr_allsub).any())
print(np.shape(res_arr_allsub))
print(np.sum(res_arr_allsub))
mean_eachsub_allsub = np.sum(res_arr_allsub) / 120
mean_ratio_allsub = mean_eachsub / (23*23*23)
print(mean_eachsub_allsub)
print(mean_ratio_allsub)

'print(np.isnan(res_arr).any())\nprint(np.shape(res_arr))\nprint(np.sum(res_arr))\nmean_eachsub = np.sum(res_arr) / 120\nmean_ratio = mean_eachsub / (23*23*23)\nprint(mean_eachsub)\nprint(mean_ratio)\n'